In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import numpy as np

/home/junsu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/junsu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/junsu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/junsu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

In [2]:
import matplotlib.pyplot as plt
from pkg.rotation_utils import *
import time

In [3]:
class RotMatLayer(layers.Layer):
    def __init__(self, axis, dim):
        self.axis = axis
        self.dim = dim
        super(RotMatLayer, self).__init__()
        if self.axis==1:
            self.r1 = tf.constant([[[1,0,0],
                                   [0,0,0],
                                   [0,0,0]]]*self.dim, dtype='float32')
            self.rc = tf.constant([[[0,0,0],
                                   [0,1,0],
                                   [0,0,1]]]*self.dim, dtype='float32')
            self.rs = tf.constant([[[0,0,0],
                                   [0,0,-1],
                                   [0,1,0]]]*self.dim, dtype='float32')
        elif self.axis==2:
            self.r1 = tf.constant([[[0,0,0],
                                   [0,1,0],
                                   [0,0,0]]]*self.dim, dtype='float32')
            self.rc = tf.constant([[[1,0,0],
                                   [0,0,0],
                                   [0,0,1]]]*self.dim, dtype='float32')
            self.rs = tf.constant([[[0,0,1],
                                   [0,0,0],
                                   [-1,0,0]]]*self.dim, dtype='float32')
        elif self.axis==3:
            self.r1 = tf.constant([[[0,0,0],
                                   [0,0,0],
                                   [0,0,1]]]*self.dim, dtype='float32')
            self.rc = tf.constant([[[1,0,0],
                                   [0,1,0],
                                   [0,0,0]]]*self.dim, dtype='float32')
            self.rs = tf.constant([[[0,-1,0],
                                   [1,0,0],
                                   [0,0,0]]]*self.dim, dtype='float32')
        else:
            raise(RuntimeError("axis should be in [1,2,3]"))
        
    # 변수를 만듭니다.
    def build(self, input_shape):
        pass

    # call 메서드가 그래프 모드에서 사용되면
    # training 변수는 텐서가 됩니다.
    @tf.function
    def call(self, input):
        return self.r1 + self.rc*K.reshape(K.cos(input), (-1,1,1)) + self.rs*K.reshape(K.sin(input), (-1,1,1))
    


In [4]:
class RotZYXLayer(layers.Layer):
    def __init__(self, dim):
        super(RotZYXLayer, self).__init__()
        self.Rz = RotMatLayer(3, dim)
        self.Ry = RotMatLayer(2, dim)
        self.Rx = RotMatLayer(1, dim)
        
    # 변수를 만듭니다.
    def build(self, input_shape):
        pass

    # call 메서드가 그래프 모드에서 사용되면
    # training 변수는 텐서가 됩니다.
    @tf.function
    def call(self, inputs):
        Rz = self.Rz(inputs[:,0])
        Ry = self.Ry(inputs[:,1])
        Rx = self.Rx(inputs[:,2])
        return tf.matmul(tf.matmul(Rz,Ry),Rx)

In [5]:
   
class LinkInfo:
    def __init__(self, Toff, axis, ltype, lname, parent):
        self.Toff, self.axis, self.ltype, self.lname, self.parent = \
            Toff, axis, ltype, lname, parent
    
    def get_tuple(self):
        return self.Toff, self.axis, self.ltype, self.lname, self.parent

In [6]:
class JointMatLayer(layers.Layer):
    def __init__(self, axis, dim, ltype, *args, **kwargs):
        self.axis = axis
        self.dim = dim
        self.ltype = ltype
        super(JointMatLayer, self).__init__(*args, **kwargs)
        if self.ltype=='revolute':
            if self.axis==1:
                self.r1 = tf.constant([[[1,0,0,0],
                                       [0,0,0,0],
                                       [0,0,0,0],
                                       [0,0,0,1]]]*self.dim, dtype='float32')
                self.rc = tf.constant([[[0,0,0,0],
                                       [0,1,0,0],
                                       [0,0,1,0],
                                       [0,0,0,0]]]*self.dim, dtype='float32')
                self.rs = tf.constant([[[0,0,0,0],
                                       [0,0,-1,0],
                                       [0,1,0,0],
                                       [0,0,0,0]]]*self.dim, dtype='float32')
            elif self.axis==2:
                self.r1 = tf.constant([[[0,0,0,0],
                                       [0,1,0,0],
                                       [0,0,0,0],
                                       [0,0,0,1]]]*self.dim, dtype='float32')
                self.rc = tf.constant([[[1,0,0,0],
                                       [0,0,0,0],
                                       [0,0,1,0],
                                       [0,0,0,0]]]*self.dim, dtype='float32')
                self.rs = tf.constant([[[0,0,1,0],
                                       [0,0,0,0],
                                       [-1,0,0,0],
                                       [0,0,0,0]]]*self.dim, dtype='float32')
            elif self.axis==3:
                self.r1 = tf.constant([[[0,0,0,0],
                                       [0,0,0,0],
                                       [0,0,1,0],
                                       [0,0,0,1]]]*self.dim, dtype='float32')
                self.rc = tf.constant([[[1,0,0,0],
                                       [0,1,0,0],
                                       [0,0,0,0],
                                       [0,0,0,0]]]*self.dim, dtype='float32')
                self.rs = tf.constant([[[0,-1,0,0],
                                       [1,0,0,0],
                                       [0,0,0,0],
                                       [0,0,0,0]]]*self.dim, dtype='float32')
            else:
                raise(RuntimeError("axis should be in [1,2,3]"))
        elif self.ltype=='prismatic':
            self.r1 = tf.constant([[[1,0,0,0],
                                   [0,1,0,0],
                                   [0,0,1,0],
                                   [0,0,0,1]]]*self.dim, dtype='float32')
            if self.axis==1:
                self.rq = tf.constant([[[0,0,0,1],
                                       [0,0,0,0],
                                       [0,0,0,0],
                                       [0,0,0,0]]]*self.dim, dtype='float32')
            elif self.axis==2:
                self.rq = tf.constant([[[0,0,0,0],
                                       [0,0,0,1],
                                       [0,0,0,0],
                                       [0,0,0,0]]]*self.dim, dtype='float32')
            elif self.axis==3:
                self.rq = tf.constant([[[0,0,0,0],
                                       [0,0,0,0],
                                       [0,0,0,1],
                                       [0,0,0,0]]]*self.dim, dtype='float32')
            else:
                raise(RuntimeError("axis should be in [1,2,3]"))
        elif self.ltype == 'fixed':
            self.r1 = tf.constant([[[1,0,0,0],
                                   [0,1,0,0],
                                   [0,0,1,0],
                                   [0,0,0,1]]]*self.dim, dtype='float32')
        else:
            raise(RuntimeError("link type {} not implemented".format(self.ltype)))
        
    # 변수를 만듭니다.
    def build(self, input_shape):
        pass

    # call 메서드가 그래프 모드에서 사용되면
    # training 변수는 텐서가 됩니다.
    @tf.function
    def call(self, input=None):
        if self.ltype=='revolute':
            return self.r1 + self.rc*K.reshape(K.cos(input), (-1,1,1)) + self.rs*K.reshape(K.sin(input), (-1,1,1))
        elif self.ltype=='prismatic':
            return self.r1 + self.rq*K.reshape(input, (-1,1,1))
        elif self.ltype=='fixed':
            return self.r1

In [7]:
class LinkLayer(layers.Layer):
    def __init__(self, link_info, dim, *args, **kwargs):
        if "name" not in kwargs:
            kwargs["name"] = link_info.lname
        super(LinkLayer, self).__init__(*args, **kwargs)
        self.link_info = link_info
        self.Toff = tf.constant(link_info.Toff)
        self.axis = link_info.axis
        self.parent = link_info.parent
        self.ltype = link_info.ltype
        self.lname = link_info.lname
        self.dim = dim
        self.Tjnt = JointMatLayer(axis=self.axis, dim=self.dim, ltype=self.ltype, name="{}_joint".format(kwargs["name"]))
        
        if self.ltype == 'fixed':
            self.q = tf.constant([0.0]*self.dim, name="{}_q".format(kwargs["name"]))
        else:
            self.q = tf.Variable([0.0]*self.dim, name="{}_q".format(kwargs["name"]))
        
    # 변수를 만듭니다.
    def build(self, input_shape):
        pass

    # call 메서드가 그래프 모드에서 사용되면
    # training 변수는 텐서가 됩니다.
    @tf.function
    def call(self, inputs):
        Tpre = inputs
        Tjnt = self.Tjnt(self.q)
        Tpo = tf.matmul(Tpre,self.Toff)
        Tcur = tf.matmul(Tpo,Tjnt)
        return Tcur

In [8]:
class RobotLayer(layers.Layer):
    def __init__(self, link_info_list, rname, base_name, dim, *args, **kwargs):
        if "name" not in kwargs:
            kwargs["name"] = rname
        super(RobotLayer, self).__init__(*args, **kwargs)
        self.link_info_list = link_info_list
        self.link_list = []
        self.rname = rname
        self.dim = dim
        self.base_name = base_name
        for link_info in self.link_info_list:
            self.link_list += [LinkLayer(link_info, dim=self.dim, name="{}_{}".format(rname,link_info.lname))]
        self.len_Q = len(self.link_list)
        
    def assign_Q(self, Q):
        for i_q in range(self.len_Q):
            link = self.link_list[i_q]
            if link.ltype != 'fixed':
                link.q.assign(Q[:,i_q])
    
    def get_Q(self):
        Q = []
        for link in self.link_list:
            Q += [link.q]
        Q = K.stack(Q, axis=1)#, name="{}_Q".format(self.rname))
        return Q
        
    # 변수를 만듭니다.
    def build(self, input_shape):
        pass

    # call 메서드가 그래프 모드에서 사용되면
    # training 변수는 텐서가 됩니다.
    @tf.function
    def call(self, inputs):
        T_dict = {self.base_name: inputs}
        T_list = []
        for link in self.link_list:
            Tcur = link(T_dict[link.parent])
            T_dict[link.lname] = Tcur
            T_list += [Tcur]
        T_list = K.stack(T_list, axis=1)
        return T_list

In [9]:
from urdf_parser_py.urdf import URDF
from pkg.ur10 import URDF_PATH, JOINT_NAMES, LINK_NAMES, ZERO_JOINT_POSE
from pkg.joint_utils import *
urdf_content = URDF.from_xml_file(URDF_PATH)

In [10]:
link_names = LINK_NAMES
base_name = LINK_NAMES[0]
joint_names = JOINT_NAMES

In [11]:
def get_link_info_list(link_names, base_name, urdf_content):
    joint_names = list(urdf_content.joint_map.keys())
    len_joints = len(joint_names)
    zero_joint_dict = joint_list2dict([0]*len_joints, joint_names)
    link_info_list = []
    for lname in link_names:
        if lname == base_name:
            continue
        jname = get_parent_joint(lname, urdf_content)
        parent_name = get_parent_link(lname, urdf_content)
        joint= urdf_content.joint_map[jname]
        link_info_list += [
            LinkInfo(
                Toff=get_tf(
                    lname, zero_joint_dict, urdf_content=urdf_content, 
                    from_link=parent_name).astype(np.float32),
                axis=np.where(joint.axis)[0][0]+1 if joint.axis is not None else 3,
                ltype=joint.type,
                lname=lname,
                parent=parent_name
            )
        ]
    return link_info_list

In [12]:
link_info_list = get_link_info_list(link_names,base_name, urdf_content)

In [13]:
# N_r = 100
# DOF = 6
# robot = RobotLayer(link_info_list, rname = "rb1", base_name=base_name, dim=N_r)

In [14]:
# qvec = []
# Pvec_list = []
# time_list = []
# asg_time_list = []
# for _ in range(100):
#     q = np.array([(0,)+tuple((np.random.rand(DOF)*2-1)*np.pi/3)+(0,) for _ in range(N_r)], dtype=np.float32)
#     qvec += [q]
#     t0 = time.time()
#     robot.assign_Q(q)
#     t01 = time.time()
#     T_all = robot([np.identity(4,dtype=np.float32)]*N_r)
#     t1 = time.time()
#     Pvec = np.array(T_all[:,:,:3,3])
#     Pvec_list += list(Pvec)
#     time_list += [t1-t0]
#     asg_time_list += [t01-t0]
# Pvec_list=np.array(Pvec_list)
# print("assign average: {} ms".format(np.mean(asg_time_list)*1000))
# print("total average: {} ms".format(np.mean(time_list)*1000))
# # for Pvec in Pvec_list:
# #     plt.plot(Pvec[:,0],Pvec[:,1],'-o')
# # plt.axis('equal')

# Visualize

In [15]:
from pkg.ros_rviz import *
pub, joints, rate = get_publisher(JOINT_NAMES)

Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


In [16]:
# pose_list = q[:,1:7]

In [17]:
# show_motion(pose_list, [], pub, joints, joint_names, error_skip=1e-6, period=1e-6)

# Constraint

In [18]:
class JointConstraintLoss(layers.Layer):
    def __init__(self, robot, *args, **kwargs):
        if "name" not in kwargs:
            kwargs["name"] = "{}_{}".format(robot.rname, "jc")
        super(JointConstraintLoss, self).__init__(*args, **kwargs)
        self.robot = robot
        
    # 변수를 만듭니다.
    def build(self, input_shape):
        pass

    # call 메서드가 그래프 모드에서 사용되면
    # training 변수는 텐서가 됩니다.
    @tf.function
    def call(self, inputs):
        Qtar = inputs[0] # (NxM)
        bAct = inputs[1] # (N)
        return K.sum(bAct * K.sum(K.square((Qtar - self.robot.get_Q())), axis=1))

In [19]:
# jc_loss = JointConstraintLoss(robot)

In [20]:
# jl = jc_loss((np.zeros((N_r, robot.len_Q), dtype='float32'), np.ones(N_r, dtype='float32')))

# Graph

In [21]:
def clip_gradient(graidents, grad_max=1):
    mx_grad = K.max(gradients)
    if mx_grad>grad_max:
        for i_g in range(len(gradients)):
            gradients[i_g] /= mx_grad
    return gradients

In [22]:
class RobotInfo:
    def __init__(self, link_info_list, rname, base_name):
        self.link_info_list, self.rname, self.base_name = \
            link_info_list, rname, base_name

In [23]:
class GraphModel(tf.keras.Model):

    def __init__(self, robot_info_list, N_sim):
        super(GraphModel, self).__init__()
        self.robot_list = []
        self.jc_list = []
        self.robot_base_list = []
        for robot_info in robot_info_list:
            robot = RobotLayer(
                robot_info.link_info_list, rname = robot_info.rname, base_name=robot_info.base_name, dim=N_sim)
            self.robot_list += [robot]
            self.jc_list += [JointConstraintLoss(robot)]
            self.robot_base_list += [[np.identity(4,dtype=np.float32)]*N_sim]
            
    def assign_Q_list(self, Q_list):
        for robot, Q in zip(self.robot_list, Q_list):
            robot.assign_Q(Q)
            
    def get_Q_list(self):
        Q_list = []
        for robot in self.robot_list:
            Q_list += [robot.get_Q()]
        return Q_list

    def call(self, inputs_list):
        T_all_list = []
        jc_list = []
        for robot, robot_base, jc_loss, inputs in zip(
                self.robot_list, self.robot_base_list, self.jc_list, inputs_list):
            T_all = robot(robot_base)
            T_all_list += [T_all]
            jc_list += [jc_loss(inputs)]
            
        return K.sum(jc_list)

    

In [46]:
N_sim = 50
N_joints = 8
DOF = 6
graph = GraphModel([RobotInfo(link_info_list, rname = "rbt1", base_name=base_name)], N_sim=N_sim)

In [47]:
learning_rate = 5e-2
# Stochastic gradient descent optimizer.
# optimizer = tf.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.99, amsgrad=False)
# optimizer = tf.optimizers.Adagrad(learning_rate*5)
optimizer = tf.optimizers.SGD(learning_rate)
# optimizer = tf.optimizers.RMSprop(learning_rate)

In [52]:
Q_ = np.array([(0,)+tuple((np.random.rand(DOF)*2-1)*np.pi)+(0,) for _ in range(N_sim)], dtype=np.float32)
graph.assign_Q_list([Q_])
time_vec = []
Q_list_vec = [list(map(lambda x: x.numpy(), graph.get_Q_list()))]
N_iter = 50
# Run training for the given number of steps.
for _ in range(N_iter):
    # Run the optimization to update W and b values.
    t1 = time.time()
    with tf.GradientTape() as g:
        # Forward pass.
        loss = graph([(np.zeros((N_sim, N_joints), dtype='float32'), np.ones(N_sim, dtype='float32'))])
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = graph.trainable_variables

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    gradients = clip_gradient(gradients, 1)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    t2 = time.time()
    time_vec += [t2-t1]
    Q_list_vec += [list(map(lambda x: x.numpy(), graph.get_Q_list()))]
print("mean {} ms".format(round(np.mean(time_vec)*1000,2)))
print("total {} ms".format(round(np.sum(time_vec)*1000,2)))

mean 6.84 ms
total 342.05 ms


In [53]:
i_rbt = 0
for i_sim in range(1): # N_sim):
    for i_iter in range(N_iter):
        q = Q_list_vec[i_iter][i_rbt][i_sim]
        pose_list = [q[1:7]]
        show_motion(pose_list, [], pub, joints, joint_names, error_skip=1e-6, period=1e-6)
        time.sleep(1e-1)

In [54]:
np.rad2deg(pose_list)

array([[ 0.0678316 , -0.0523061 , -0.11742061, -0.12202853, -0.05438674,
         0.03700944]], dtype=float32)

In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# MNIST dataset parameters.
num_classes = 10 # total classes (0-9 digits).
num_features = 784 # data features (img shape: 28*28).

# Training parameters.
learning_rate = 0.1
training_steps = 2000
batch_size = 50
display_step = 100

# Network parameters.
n_hidden_1 = 128 # 1st layer number of neurons.
n_hidden_2 = 256 # 2nd layer number of neurons.

In [ ]:
# Prepare MNIST data.
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

In [ ]:

# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [ ]:

# Create TF Model.
class NeuralNet(Model):
    # Set layers.
    def __init__(self):
        super(NeuralNet, self).__init__()
        # First fully-connected hidden layer.
        self.fc1 = layers.Dense(n_hidden_1, activation=tf.nn.relu)
        # First fully-connected hidden layer.
        self.fc2 = layers.Dense(n_hidden_2, activation=tf.nn.relu)
        # Second fully-connecter hidden layer.
        self.out = layers.Dense(num_classes)

    # Set forward pass.
    def call(self, x, is_training=False):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            x = tf.nn.softmax(x)
        return x

# Build neural network model.
neural_net = NeuralNet()

In [ ]:

# Cross-Entropy Loss.
# Note that this will apply 'softmax' to the logits.
def cross_entropy_loss(x, y):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # Average loss across the batch.
    return tf.reduce_mean(loss)

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

In [ ]:
# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        # Forward pass.
        pred = neural_net(x, is_training=True)
        # Compute loss.
        loss = cross_entropy_loss(pred, y)
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = neural_net.trainable_variables

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [ ]:
import time

In [ ]:
time_vec = []
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    t1 = time.time()
    run_optimization(batch_x, batch_y)
    t2 = time.time()
    time_vec += [t2-t1]
    
    if step % display_step == 0:
        pred = neural_net(batch_x, is_training=True)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

In [ ]:
np.mean(time_vec)*100